In [ ]:
# build_engine.py
import tensorrt as trt
import os

# --- 설정 변수 ---
ONNX_MODEL_PATH = "resources/models/yolov8_custom_fixed_v2.onnx"
ENGINE_PATH = "resources/models/yolov8_custom_v2.engine" # 생성될 엔진 파일 경로
INPUT_NAME = "images"  # ONNX 모델의 입력 레이어 이름 (대부분 'images' 또는 'input')
                        # onnx.load(ONNX_MODEL_PATH).graph.input[0].name 으로 확인 가능
INPUT_H = 192          # ONNX 모델이 기대하는 입력 높이 (Detector 클래스의 self.input_height와 동일)
INPUT_W = 320          # ONNX 모델이 기대하는 입력 너비 (Detector 클래스의 self.input_width와 동일)
BATCH_SIZE = 1         # 배치 크기
USE_FP16 = False       # FP16 정밀도 사용 여부 (GPU가 지원해야 함)
# ---

TRT_LOGGER = trt.Logger(trt.Logger.WARNING) # 로깅 레벨 (INFO, WARNING, ERROR, VERBOSE 등)

def build_engine(onnx_file_path, engine_file_path, input_name, input_h, input_w, batch_size=1, use_fp16=False):
    if os.path.exists(engine_file_path):
        print(f"TensorRT 엔진 파일이 이미 존재합니다: {engine_file_path}")
        return engine_file_path

    builder = trt.Builder(TRT_LOGGER)
    
    # EXPLICIT_BATCH 플래그로 네트워크 생성
    network_flags = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
    network = builder.create_network(network_flags)
    
    parser = trt.OnnxParser(network, TRT_LOGGER)
    config = builder.create_builder_config()

    # 작업 공간 메모리 설정 (필요에 따라 조절)
    # config.max_workspace_size = 1 << 30  # 1GB (Deprecated in TRT 8.x+)
    config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30) # 1GB

    if use_fp16:
        if builder.platform_has_fast_fp16:
            config.set_flag(trt.BuilderFlag.FP16)
            print("FP16 모드가 활성화되었습니다.")
        else:
            print("FP16이 현재 플랫폼에서 지원되지 않습니다. FP32로 빌드합니다.")

    # ONNX 파일 로드 및 파싱
    print(f"ONNX 파일 파싱 시작: {onnx_file_path}")
    with open(onnx_file_path, 'rb') as model:
        if not parser.parse(model.read()):
            print('오류: ONNX 파일 파싱에 실패했습니다.')
            for error in range(parser.num_errors):
                print(parser.get_error(error))
            return None
    print("ONNX 파일 파싱 완료.")

    # 입력 텐서의 동적 형태를 사용하지 않는 경우, 고정 크기 설정
    # 이미 EXPLICIT_BATCH로 생성했으므로, ONNX 모델 내의 배치 차원이 사용됨
    # network.get_input(0).shape = [batch_size, 3, input_h, input_w] # 모델에 따라 입력 이름/인덱스 확인 필요
    # 명시적으로 입력 이름을 사용하는 것이 더 안전
    if network.num_inputs == 0:
        print("오류: ONNX 모델에서 입력을 찾을 수 없습니다.")
        return None
    
    # 모델의 첫 번째 입력에 대해 shape 설정 (필요 시, ONNX에 이미 정의되어 있다면 생략 가능)
    # model_input = network.get_input(0) # 첫 번째 입력으로 가정
    # model_input.shape = [batch_size, 3, input_h, input_w] # BCHW

    print(f"TensorRT 엔진 빌드 시작: {engine_file_path}")
    # serialized_engine = builder.build_serialized_network(network, config) # 최신 TRT
    # 구버전 호환성을 위해 build_engine 후 serialize
    engine = builder.build_engine(network, config)

    if not engine:
        print("오류: TensorRT 엔진 빌드에 실패했습니다.")
        return None
    
    print("TensorRT 엔진 빌드 완료.")

    # 엔진 직렬화 및 파일 저장
    serialized_engine = engine.serialize()
    with open(engine_file_path, "wb") as f:
        f.write(serialized_engine)
    
    print(f"TensorRT 엔진 저장 완료: {engine_file_path}")

    # 리소스 정리
    del engine
    del network
    del config
    del parser
    del builder
    
    return engine_file_path

if __name__ == '__main__':
    # ONNX 모델의 실제 입력 이름을 확인하고 INPUT_NAME을 설정하세요.
    # (예: Netron으로 ONNX 모델을 열어 입력 노드의 이름 확인)
    # onnx_model = onnx.load(ONNX_MODEL_PATH)
    # input_name_from_onnx = onnx_model.graph.input[0].name
    # print(f"ONNX 모델의 입력 이름: {input_name_from_onnx}") # 이 값을 INPUT_NAME에 사용
    
    build_engine(ONNX_MODEL_PATH, ENGINE_PATH, INPUT_NAME, INPUT_H, INPUT_W, BATCH_SIZE, USE_FP16)